In [ ]:
import random
import json

import torch
import import_ipynb

#from model import NeuralNet
from ipynb.fs.full.model import NeuralNet
from ipynb.fs.full.nltk_utils import bag_of_words, tokenize
#from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents (1).json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()


bot_name = "GRIET"
#print("GRIET :HI! I am your personal bot. I am here to answer queries on GRIET (type 'quit' to exit)")

def get_response(msg):
    
    sentence = tokenize(msg)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                return random.choice(intent['responses'])
    
    return "I do not understand..."


if __name__ == "__main__":
    print("GRIET: HI! I am GRIET bot.Let's chat! (type 'quit' to exit)")
    while True:
        
        sentence = input("You: ")
        if sentence == "quit": 
            break

        resp = get_response(sentence)
        print("GRIET:", resp)
        bot_speaking(resp) 
        
    
    
    
    
    
    
    
    
    

GRIET: HI! I am GRIET bot.Let's chat! (type 'quit' to exit)
You: hi
GRIET: Hello, thanks for visiting


In [1]:
import speech_recognition as sr
r = sr.Recognizer()
import pyttsx3
engine = pyttsx3.init()

In [2]:
def bot_speaking(message):
    engine.say(message)
    engine.runAndWait()
    if engine._inLoop:
        engine.endLoop()

In [ ]:
def get_input():
    with sr.Microphone() as source:
       #print("Say something!")
        bot_speaking("Hey mate say something")
        audio = r.listen(source, timeout = 0)
       #print("perfect, Thanks")
        bot_speaking("Perfect, Thanks")
    try:
        msg = r.recognize_google(audio)
        print("TEXT: "+msg);
        bot_speaking("you said:"+msg)
        return msg
    except:
     # print("Dude it's not working:")
      bot_speaking("Sorry mate! It's not working")
      pass;